In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel("Сводная_ведомость_учета_успеваемости_класса.xlsx", header=2)
df.tail()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
187,NaN,11,1 Полугодие,5,4,5,5,5,5,5,...,5,5,NaN,NaN,5,5,5,5,NaN,5
188,NaN,NaN,2 Полугодие,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
189,NaN,NaN,Год,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,NaN,NaN,Средняя оценка,5,"4,5",5,5,"4,75",5,5,...,5,5,0,5,5,5,5,"4,8",0,5
191,NaN,NaN,Итог. оценка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Определение столбцов
subjects = [
    "Алг и матанализ", "Биол.", "Вер и Стат", "Геогр.", "Геом.", 
    "Англ.яз", "Инф.", "Ист.", "Лит-ра", "Обществ.", "ОБиЗР", 
    "Практ. по матем.", "Практ. по обществ.", "Рус.яз.", "Совр. литерат", 
    "Физика", "Физ-ра", "Фин.гр.", "Химия"
]

# Создание уникальных ID для учеников
student_names = df[df.iloc[:, 0].str.contains("Ученик|Ученица", na=False)].iloc[:, 0]
student_ids = {name: i+1 for i, name in enumerate(student_names)}

In [7]:
# Сбор данных
data = []
current_id = 0
current_class = None

def row_to_float(row):
    grades = row[3:22].values
    grades = [str(x).replace(',', '.') for x in grades]
    grades = list(map(float, grades))
    return grades
    
for idx, row in df.iterrows():
    if "Ученик" in str(row.iloc[0]) or "Ученица" in str(row.iloc[0]):
        current_name = row.iloc[0]
        current_id = student_ids[current_name]
    if pd.notna(row.iloc[1]) and (row.iloc[1] in [10, 11] or str(row.iloc[1]).isdigit()):
        current_class = int(row.iloc[1])
    if isinstance(row.iloc[2], str):
        if "Полугодие" in row.iloc[2] or "Год" in row.iloc[2]:
            period = row.iloc[2]
            grades = row_to_float(row)
            grades = list(map(float, grades))
            data.append([current_id, current_class, period] + list(grades))
        elif "Средняя оценка" in row.iloc[2]:
            period = "Средняя оценка"
            grades = row_to_float(row)
            data.append([current_id, "", period] + list(grades))
        elif "Итог. оценка" in row.iloc[2]:
            period = "Итог. оценка"
            grades = row_to_float(row)
            data.append([current_id, "", period] + list(grades))

In [9]:
result = pd.DataFrame(
    data,
    columns=["ID", "Класс", "Период"] + subjects
)
# # Замена пустых значений и преобразование типов
result = result.replace("", np.nan)
#result[subjects] = result[subjects].apply(pd.to_numeric, errors='coerce').astype('float64')

# # Сохранение в CSV
result.to_csv("успеваемость.csv", index=False, encoding="utf-8-sig")
result.head(10)

C:\Users\motve\AppData\Local\Temp\ipykernel_22488\2268424234.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = result.replace("", np.nan)


,ID,Класс,Период,Алг и матанализ,Биол.,Вер и Стат,Геогр.,Геом.,Англ.яз,Инф.,...,Обществ.,ОБиЗР,Практ. по матем.,Практ. по обществ.,Рус.яз.,Совр. литерат,Физика,Физ-ра,Фин.гр.,Химия
0,1,10.0,1 Полугодие,4.0,4.0,4.0,5.0,4.00,5.0,NaN,...,4.00,5.0,NaN,5.0,4.0,5.0,4.0,5.0,NaN,4.00
1,1,10.0,2 Полугодие,4.0,4.0,5.0,5.0,3.00,5.0,NaN,...,4.00,5.0,NaN,5.0,4.0,5.0,4.0,4.0,NaN,4.00
2,1,10.0,Год,4.0,4.0,5.0,5.0,4.00,5.0,NaN,...,4.00,5.0,NaN,5.0,4.0,5.0,4.0,5.0,NaN,4.00
3,1,11.0,1 Полугодие,4.0,4.0,4.0,5.0,4.00,5.0,5.0,...,5.00,5.0,NaN,NaN,4.0,5.0,4.0,5.0,NaN,5.00
4,1,11.0,2 Полугодие,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
5,1,11.0,Год,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,NaN,Средняя оценка,4.0,4.0,4.5,5.0,3.75,5.0,5.0,...,4.25,5.0,0.0,5.0,4.0,5.0,4.0,4.8,0.0,4.25
7,1,NaN,Итог. оценка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,10.0,1 Полугодие,5.0,5.0,5.0,5.0,5.00,5.0,NaN,...,5.00,5.0,NaN,5.0,5.0,5.0,5.0,5.0,NaN,5.00
9,2,10.0,2 Полугодие,5.0,5.0,5.0,5.0,5.00,5.0,NaN,...,5.00,5.0,NaN,5.0,5.0,5.0,5.0,5.0,NaN,5.00
